In [29]:
import cv2 as cv
import numpy as np
import os
import sys
import tensorflow as tf
from sklearn.model_selection import train_test_split

In [58]:
EPOCHS = 10
IMG_WIDTH = 30
IMG_HEIGHT = 30
NUM_CATEGORIES = 43
TEST_SIZE = 0.4
filename = "traffic.h5"
data_dir = r"traffic_signs"

In [31]:
def load_data(data_dir):
   # loading data and labels
    images = []
    labels = []
    
    # joining data_dir path and folder paths
    for dir in os.listdir(data_dir):
        folder = os.path.join(data_dir, dir)
        
        # message
        if os.path.isdir(folder):
            print(f"Loading files from {folder}")
        
        # looking for images in folders
        for file in os.listdir(folder):
            # reading and resizing images
            img = cv.imread(os.path.join(folder, file))
            img = cv.resize(img, (IMG_WIDTH, IMG_HEIGHT))
            
            # appending lists
            images.append(img)
            labels.append(int(dir))
            
    # returning lists        
    return images, labels

In [53]:
def get_model():
    # defining model
    model = tf.keras.models.Sequential([
        
        # adding convolutional layer with 32 filters by 3x3 kernal
        tf.keras.layers.Conv2D(32, (3,3), activation="sigmoid", input_shape=(IMG_WIDTH, IMG_HEIGHT, 3)),
        
        # adding max_pooling layer with pool size 2x2
        tf.keras.layers.MaxPooling2D(pool_size=(2, 2)),
        
        # flattening all the units with dropout with 30%
        tf.keras.layers.Flatten(),
        
        # adding hidden layer with 128 units with 50% dropout
        tf.keras.layers.Dense(256, activation="sigmoid"),
        tf.keras.layers.Dropout(0.5),
        
        # adding output layer with 10 units to get probablity for 10 digits with softmax activation fn
        tf.keras.layers.Dense(NUM_CATEGORIES, activation="softmax")
    ])
    
    # compiling model
    model.compile(
        optimizer="adam",
        loss="categorical_crossentropy",
        metrics=["accuracy"]
    )
    
    # getting model summary
    model.summary()
    return model

In [33]:
# getting data
images, labels = load_data(data_dir)

# converting integer labeles to binary matrices to use categorical_crossentropy loss fn
labels = tf.keras.utils.to_categorical(labels)

# performing train test split
x_train, x_test, y_train, y_test =  train_test_split(np.array(images), np.array(labels), test_size=TEST_SIZE)

Loading files from traffic_signs\0
Loading files from traffic_signs\1
Loading files from traffic_signs\10
Loading files from traffic_signs\11
Loading files from traffic_signs\12
Loading files from traffic_signs\13
Loading files from traffic_signs\14
Loading files from traffic_signs\15
Loading files from traffic_signs\16
Loading files from traffic_signs\17
Loading files from traffic_signs\18
Loading files from traffic_signs\19
Loading files from traffic_signs\2
Loading files from traffic_signs\20
Loading files from traffic_signs\21
Loading files from traffic_signs\22
Loading files from traffic_signs\23
Loading files from traffic_signs\24
Loading files from traffic_signs\25
Loading files from traffic_signs\26
Loading files from traffic_signs\27
Loading files from traffic_signs\28
Loading files from traffic_signs\29
Loading files from traffic_signs\3
Loading files from traffic_signs\30
Loading files from traffic_signs\31
Loading files from traffic_signs\32
Loading files from traffic_signs

In [54]:
# getting model
model = get_model()

Model: "sequential_12"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_19 (Conv2D)          (None, 28, 28, 32)        896       
                                                                 
 max_pooling2d_19 (MaxPoolin  (None, 14, 14, 32)       0         
 g2D)                                                            
                                                                 
 flatten_12 (Flatten)        (None, 6272)              0         
                                                                 
 dense_36 (Dense)            (None, 256)               1605888   
                                                                 
 dropout_29 (Dropout)        (None, 256)               0         
                                                                 
 dense_37 (Dense)            (None, 43)                11051     
                                                     

In [55]:
# training model
model.fit(x_train, y_train, epochs=EPOCHS)

Epoch 1/10
500/500 [==============================] - 8s 16ms/step - loss: 2.3534 - accuracy: 0.3926
Epoch 2/10
500/500 [==============================] - 7s 13ms/step - loss: 0.4848 - accuracy: 0.8920
Epoch 3/10
500/500 [==============================] - 7s 13ms/step - loss: 0.2164 - accuracy: 0.9566
Epoch 4/10
500/500 [==============================] - 6s 13ms/step - loss: 0.1342 - accuracy: 0.9742
Epoch 5/10
500/500 [==============================] - 7s 14ms/step - loss: 0.0907 - accuracy: 0.9835
Epoch 6/10
500/500 [==============================] - 7s 15ms/step - loss: 0.0672 - accuracy: 0.9872
Epoch 7/10
500/500 [==============================] - 7s 14ms/step - loss: 0.0529 - accuracy: 0.9902
Epoch 8/10
500/500 [==============================] - 7s 14ms/step - loss: 0.0479 - accuracy: 0.9911
Epoch 9/10
500/500 [==============================] - 7s 13ms/step - loss: 0.0477 - accuracy: 0.9894
Epoch 10/10
500/500 [==============================] - 7s 14ms/step - loss: 0.0383 - accura

In [56]:
# Evaluating the model
model.evaluate(x_test,  y_test, verbose=2)

333/333 - 2s - loss: 0.0589 - accuracy: 0.9865 - 2s/epoch - 6ms/step


[0.05886464565992355, 0.9864864945411682]

In [60]:
# saving the model
model.save(filename)
print(f"Model saved to {filename}.")

Model saved to traffic.h5.
